In [2]:
import os
import pandas as pd
import cv2

In [3]:
# Set csv path
csv1_path = '../Geminiden/kam1.csv'
csv2_path = '../Geminiden/kam2.csv'


path_c1 = "/media/disk1/KILabDaten/Geminiden2021/Kamera1/"
path_c2 = "/media/disk1/KILabDaten/Geminiden2021/Kamera2/"
fileend = '.mov'

outpath_c1 = path_c1 + "MeteorFrames/"
outpath_c2 = path_c2 + "MeteorFrames/"

# Chose which cam to use

In [4]:
camnum = 2

In [5]:
camdict = {1:[csv1_path, path_c1, outpath_c1], 2:[csv2_path, path_c2, outpath_c2]}

file_path = camdict[camnum][1] #path_c1
csv_path = camdict[camnum][0] #csv1_path
outpath = camdict[camnum][2] #outpath_c1

if not os.path.isdir(outpath):
  os.makedirs(outpath)

# Run the script

In [20]:
df = pd.read_csv(csv_path, sep=';', encoding='latin-1')
df['CTL'] = pd.to_datetime(df['CTL'].str.strip(), format='%H:%M:%S')#.dt.time

df['CTLend'] = df['CTL'] + pd.Timedelta(seconds=4)
df.CTL = df.CTL#.dt.time
df.CTLend = df.CTLend#.dt.time

df['CTL'] = df['CTL'].dt.hour + df['CTL'].dt.minute*60 + df['CTL'].dt.second
df['CTLend'] = df['CTLend'].dt.hour + df['CTLend'].dt.minute*60 + df['CTLend'].dt.second

print(df.head())

   Nr.              Dateiname  CTL  H geschätzt mag Pos.  GEM  Sonst.  \
0    1  NINJA3_S001_S001_T001   46               -1   ol  1.0     NaN   
1    2  NINJA3_S001_S001_T001  160                2   ur  NaN     1.0   
2    3  NINJA3_S001_S001_T001  181                4    m  NaN     1.0   
3    4  NINJA3_S001_S001_T001  279                4   um  1.0     NaN   
4    5  NINJA3_S001_S001_T001  279                1   um  1.0     NaN   

  Bemerkungen  CTLend  
0         NaN      50  
1         NaN     164  
2         NaN     185  
3         NaN     283  
4         NaN     283  


In [21]:
col_data_name = df['Dateiname'].tolist()
col_start = df['CTL'].tolist()
col_end = df['CTLend'].tolist()
col_num = df['Nr.']

In [22]:
#return video length in seconds
def video_length(filename):
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    print(filename)
    duration = frame_count/fps

    video.release()
    return int(duration), frame_count, fps

#d = video_length("D:\Geminiden 2021/Kamera 1/NINJA2_S001_S001_T007.MOV")
#print(d)

In [23]:
name_time_id_dict = {}

for name, start, id in zip(col_data_name, col_start, col_num):
    if name not in name_time_id_dict:
        name_time_id_dict[name] = []
    
    
    name_time_id_dict[name].append([id, start])

In [24]:
n_dict = {}

for file in name_time_id_dict:
    fdir = file_path + file + '.MOV'
    l, framecount, _ = video_length(fdir)
    n_dict[file] = []
    for x in name_time_id_dict[file]:
        i, s = x
        nstart = s*25-5*25
        nend = s*25+5*25
        if nstart<1:
            nstart = 1
        if nend > int(framecount):
            nend = int(framecount)
        n_dict[file].append([i, nstart, nend])

/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T001.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T005.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T006.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T009.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T012.MOV


In [26]:
frame_to_id_dict= {}


for file in n_dict:
    fdir = file_path + file + '.MOV'
    l, framecount, _ = video_length(fdir)

    frame_to_id_dict[file] = {}

    ids = [i[0] for i in n_dict[file]]
    starttimes = [i[1] for i in n_dict[file]]
    endtimes = [i[2] for i in n_dict[file]]

    for id, s, e in zip(ids,starttimes, endtimes):
        for j in range(s,e+1):
            if j not in frame_to_id_dict[file]:
                frame_to_id_dict[file][j]=[]
            if id not in frame_to_id_dict[file][j]:
                frame_to_id_dict[file][j].append(id)



/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T001.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T005.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T006.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T009.MOV
/media/disk1/KILabDaten/Geminiden2021/Kamera2/NINJA3_S001_S001_T012.MOV


In [27]:
print(len(df.loc[df['Dateiname'] == 'NINJA2_S001_S001_T009']))

0


In [ ]:
ignore_list = ['NINJA2_S001_S001_T007']

In [50]:
for file in frame_to_id_dict:
    fdir = file_path + file + '.MOV'
    l, framecount, _ = video_length(fdir)


    cap = cv2.VideoCapture(fdir)
    print(file)
    if file in ignore_list:
        cap.release()
        continue


    while cap.isOpened():
        ret, frame = cap.read()
        frameid = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        if ret:
            if frameid in frame_to_id_dict[file].keys():

                for obj_id in frame_to_id_dict[file][frameid]:
                    out_object_path = outpath + str(obj_id) + '/'
                    if not os.path.isdir(out_object_path):
                        os.makedirs(out_object_path)
                    out_img_path = out_object_path + str(frameid) + '.png'
                    if not os.path.isfile(out_img_path):
                        img_rgb = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)
                        cv2.imwrite(out_img_path, img_rgb)
                


    cap.release()

NINJA2_S001_S001_T007
